# Meteo con el pato

### Craga de datos en el pato
* dataset: **meteorologia-tiempo-real** 

In [ ]:
%%capture
import duckdb

#conexion a bbdd en memoria o en fichero
#conn = duckdb.connect() 
conn = duckdb.connect('/home/jcaubin/duck_test.db') 

%load_ext sql
%sql conn --alias duckdb

#limite de filas a mostrar
%config SqlMagic.displaylimit = 100 

In [ ]:
# imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f"### hora de ejecución: {t}"))

In [ ]:
import requests

response = requests.get('https://datos.madrid.es/egob/catalogo/300392-11041819-meteorologia-tiempo-real.csv')
if response.status_code==200:
    with open('../data/300392-11041819-meteorologia-tiempo-real.csv', 'wb') as tmp_file:
        tmp_file.write(response.content)


In [ ]:
%%sql 

create or replace table meteo_today as
select * from read_csv('../data/300392-11041819-meteorologia-tiempo-real.csv');

In [ ]:

%%sql

create or replace table estaciones as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv')

In [ ]:

%%sql

create or replace table magnitudes as
select cast(CODIGO as int) CODIGO, PARAMETRO, UNIDAD, cast(COD_TECNICA as int) COD_TECNICA, TECNICA 
from read_csv('../maestras/parametros_meteo.csv')

In [ ]:
%%sql 

create or replace table td as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALOR	 from (
from meteo_today
unpivot (
    (VALOR)
for HORA in (
    H01, 
    H02, 
    H03, 
    H04, 
    H05, 
    H06, 
    H07, 
    H08, 
    H09, 
    H10, 
    H11, 
    H12, 
    H13, 
    H14, 
    H15, 
    H16, 
    H17, 
    H18, 
    H19, 
    H20, 
    H21, 
    H22, 
    H23, 
    H24 
    ) 
)
);

In [ ]:
%%sql 

create or replace  table tv as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALIDEZ	 
from (
from meteo_today
unpivot (
    (VALIDEZ)
for HORA in (
    V01, 
    V02, 
    V03, 
    V04, 
    V05, 
    V06, 
    V07, 
    V08, 
    V09, 
    V10, 
    V11, 
    V12, 
    V13, 
    V14, 
    V15, 
    V16, 
    V17, 
    V18, 
    V19, 
    V20, 
    V21, 
    V22, 
    V23, 
    V24, 
    ) 
 ))
 ;


In [ ]:
%%sql

select td.PROVINCIA, td.MUNICIPIO, td.ESTACION, td.MAGNITUD
    , concat(td.PROVINCIA, td.MUNICIPIO, td.ESTACION,  '_' ,td.MAGNITUD, '_98') PUNTO_MUESTREO, td.ANO, td.MES, td.DIA, td.H, td.VALOR 
    , tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD, m.PARAMETRO
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and  td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO
inner join magnitudes m on m.CODIGO = tv.MAGNITUD 

In [ ]:
%%sql 

create or replace table meteo_madrid24 as(
select td.PROVINCIA, td.MUNICIPIO, td.ESTACION, td.MAGNITUD
    , concat(td.PROVINCIA, td.MUNICIPIO, td.ESTACION,  '_' ,td.MAGNITUD, '_98') PUNTO_MUESTREO, td.ANO, td.MES, td.DIA, td.H, td.VALOR 
    , tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD, m.PARAMETRO
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and  td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO
inner join magnitudes m on m.CODIGO = tv.MAGNITUD 
)

In [ ]:
%%sql
select punto_muestreo, count(1)
from calair
group by all
order by all

In [ ]:
%%sql
/*MEGE de datos con la tabla principal*/
INSERT OR REPLACE INTO CALAIR BY NAME 
SELECT PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO, ANO, MES, DIA, H, VALOR, VALIDEZ, ESTACION_DESC, ALTITUD, PARAMETRO
FROM duck_test.main.meteo_madrid24;

In [ ]:
%%sql

SELECT  ANO, MES, DIA, COUNT(1) n
FROM duck_test.main.calair
GROUP BY ALL
order by all;

In [ ]:
%sql --close duckdb